In [12]:
#!pip install geopandas
#!pip install tqdm
#!pip install geopandas
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geopandas as gpd
import folium
from shapely.geometry.point import Point
from shapely.geometry import *
from folium import plugins
from folium.plugins import MousePosition
import math

In [13]:
#Selecting port
def select_port(filename):
    fin = open(filename, "r", errors='ignore')  # LNG Port
    read_p = fin.read()
    split_p = read_p.split(',')
    port_name = []
    port_coordinate = []
    port = {}
    for x in range(len(split_p)):
        p = int(x / 3)
        q = float(x / 3)
        if p == q:
            split_p[x] = split_p[x].strip('\t')
            split_p[x] = split_p[x].strip('\n')
            port_name.append(split_p[x])
            port_coordinate.append(float(split_p[x + 1]))
            port_coordinate.append(float(split_p[x + 2]))

    for y in range(int(len(split_p) / 3)):
        port[port_name[y]] = (port_coordinate[y * 2], port_coordinate[y * 2 + 1])
    
    fin.close()

    # start port, end port 입력
    print('Port name list : ', port_name)
    start_port = input('Start port를 입력하세요 : ')
    end_port = input('End port를 입력하세요 : ')
    start_lat = round(port[start_port][0], 0)
    start_lng = round(port[start_port][1], 0)
    end_lat = round(port[end_port][0], 0)
    end_lng = round(port[end_port][1], 0)

    print('start port : ', start_port)
    print('(', start_lat, ',', start_lng, ')')
    print('end port : ', end_port)
    print('(', end_lat, ',', end_lng, ')')
    a = []
    b = []
    if abs(start_lng - end_lng) > 180:
        if start_lng > end_lng:
            end_lng += 360
        elif end_lng > start_lng:
            start_lng += 360
    
    a.append(start_lat)
    a.append(start_lng)
    b.append(end_lat)
    b.append(end_lng)
    return a, b, start_port, end_port

3
start_lat: 35
start_lng: 129
end_lat: 37
end_lng: 126


In [14]:
#Selecting port using googlemaps
def select_port_googlemaps(gmaps_key):
    import googlemaps
    gmaps = googlemaps.Client(key=gmaps_key)
    start_port = input('Start를 입력하세요 : ')
    end_port = input('End를 입력하세요 : ')
    start = gmaps.geocode(start_port, language='ko')  # 출발항구 입력 - 위도 경도받기
    end = gmaps.geocode(end_port, language='ko')  # 도착항구 입력 - 위도 경도받기

    start_lat = round(start[0]['geometry']['location']['lat'], 0)
    start_lng = round(start[0]['geometry']['location']['lng'], 0)
    end_lat = round(end[0]['geometry']['location']['lat'], 0)
    end_lng = round(end[0]['geometry']['location']['lng'], 0)

    print('start : ', start_lat, start_lng)
    print('end : ', end_lat, end_lng)
    a = []
    b = []
    if abs(start_lng - end_lng) > 180:
        if start_lng > end_lng:
            end_lng += 360
        elif end_lng > start_lng:
            start_lng += 360
    a.append(start_lat)
    a.append(start_lng)
    b.append(end_lat)
    b.append(end_lng)
    return a, b, start_port, end_port

In [15]:
#Read wind data
def read_file(filename):
    fopen = open(filename, "r", errors='ignore')
    read = fopen.read()
    split = read.split()
    l = []
    l.append([])
    for x in range(180):
        for y in range(x * 360 + 360):
            l[x].append(split[y])
            l.append([])
    fopen.close()
    return l

In [18]:
#Read barriers data
def Find_barriers(filename):
    barriers = []
    fin = open("boundary.txt", "r")
    read = fin.read()
    split = read.split()
    tmp_boundary = []
    boundary = []
    for x in range(len(split)):
        tmp_boundary.append(float(split[x]))
    for y in range(len(tmp_boundary)):
        boundary.append(tmp_boundary[y])

    for z in range(len(boundary)):
        zz = z / 2
        if round(zz, 0) == zz:
            if boundary[z + 1] < -90 or boundary[z + 1] > 90:
                continue
            barriers.append((boundary[z], boundary[z + 1]))
            if boundary[z] < 0:
                barriers.append((boundary[z] + 360, boundary[z + 1]))
        else:
            continue
    barriers = list(set(barriers))
    fin.close()
    return barriers

In [19]:
#Find H (거리)
def heuristic(start, end):
    dx = abs(start[0] - end[0])
    dy = abs(start[1] - end[1])
    dist = (dx**2 + dy**2)**0.5
    return dist

KeyError: None

In [20]:
#Find G (거리)
def Find_G(current, openlsit_v):
    dx = abs(current[0] - openlsit_v[0])
    dy = abs(current[1] - openlsit_v[1])
    G = (dx ** 2 + dy ** 2) ** 0.5 * 100000000
    return G

#Find G(Wind 세기)
def G_Wind(openlist_v,u,v):
    tmp_a = int(openlist_v[0])
    tmp_b = int(openlist_v[1])
    a = tmp_a + 90
    b = tmp_b + 180
    if b > 360 or b == 360:
        b = b -360
    if a == 180:
        a = a - 180
    G = (float(u[a][b]) ** 2 + float(v[a][b]) ** 2) ** 0.5
    return G

'0.6.1'

In [66]:
#Find neighbour
def search(current):
    n = []
    for dx, dy in [(1,0),(1,0),(0,1),(0,-1),(1,1),(-1,1),(1,-1),(-1,-1)]:
        x2 = current[0] + dx
        y2 = current[1] + dy
        x2 = round(x2, 1)
        y2 = round(y2, 1)
        if x2 < -90 or y2 < -180 or x2 > 90 or y2 > 360:
            continue
        n.append((x2, y2))
    return n

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."
...,...,...,...,...,...,...
172,7111024,Europe,Serbia,SRB,101800.0,"POLYGON ((18.82982 45.90887, 18.82984 45.90888..."
173,642550,Europe,Montenegro,MNE,10610.0,"POLYGON ((20.07070 42.58863, 19.80161 42.50009..."
174,1895250,Europe,Kosovo,-99,18490.0,"POLYGON ((20.59025 41.85541, 20.52295 42.21787..."
175,1218208,North America,Trinidad and Tobago,TTO,43570.0,"POLYGON ((-61.68000 10.76000, -61.10500 10.890..."


In [22]:
#Astar algorithm
def AStarsearch(start, end, barriers,u,v):
    G = {}
    F = {}
    parents = {}

    # initializing
    G[start] = 0
    F[start] = G[start] + heuristic(start, end)
    closedlist = set()
    openlist = set([start])
    current = None

    while current != end:
        current = None
        tmp_F = None
        for pos in openlist:
            if current is None or F[pos] < tmp_F:
                tmp_F = F[pos]
                current = pos
        if end[0] - 3 < current[0] < end[0] + 3 and end[1] - 3 < current[1] < end[1] + 3:
            break

        openlist.remove(current)
        closedlist.add(current)

        for pos in search(current):
            if pos in closedlist:
                continue
            tmp_G = G[current] + Find_G(current, pos)#거리에 따른 G값
            #tmp_G = G[current] + G_Wind(pos,u,v)  # Wind power에 따른 G값

            if pos not in openlist:
                openlist.add(pos)
            elif tmp_G >= G[pos]:
                continue
            # barriers 근처에 오면 가중치를 주는 방식으로 경로 탐색(경도차가 180 이상일때 사용)
            if start[1] > 360 or end[1] > 360:
                for t in range(len(barriers)):
                    if barriers[t][0] - 1.2 <= pos[1] <= barriers[t][0] + 1.2 and barriers[t][1] - 1.2 <= pos[0] <= \
                            barriers[t][1] + 1.2:
                        if -7 < pos[1] < -3 and 35 < pos[0] < 37:
                            continue
                        elif 31 < pos[1] < 35 and 27 < pos[0] < 32:
                            continue
                        else:
                            tmp_G += 999999999
                            break

            # geopandas의 polygon 안에 현재 좌표가 들어가는지 판별해서 가중치를 주는 방식으로 경로 탐색
            countries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
            poly = []
            point = Point(pos[1], pos[0])
            for x in range(len(countries.geometry)):
                if type(countries.geometry[x]) == MultiPolygon:
                    for y in range(len(countries.geometry[x])):
                        poly.append(countries.geometry[x][y])
                else:
                    poly.append(countries.geometry[x])
            for z in range(len(poly)):
                if poly[z].contains(point):
                    if -7 < pos[1] < -3 and 35 < pos[0] < 37:
                        continue
                    elif 31 < pos[1] < 35 and 27 < pos[0] < 32:
                        continue
                    else:
                        tmp_G += 99999999999

            parents[pos] = current
            H = heuristic(pos, end)
            G[pos] = tmp_G
            F[pos] = H + G[pos]

    # 경로 역추적
    path = [current]
    while current in parents:
        current = parents[current]
        path.append(current)
    path.reverse()
    path.append(end)
    return path

{'init': 'epsg:4326'}

In [27]:
barriers = Find_barriers("boundary.txt")
start, end, start_port, end_port = select_port("LNG_Port.txt")
result = AStarsearch((start[0],start[1]),(end[0],end[1]), barriers, read_file("time1(u10).txt"), read_file("time1(v10).txt"))
print(result)
print(len(result))

In [36]:
m = folium.Map(location=[start[0], start[1]])
formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"
plugins.ScrollZoomToggler().add_to(m)
MousePosition(position='topright', separator=' | ', empty_string='NaN', lng_first=True, num_digits=20,
                  prefix='Coordinates:', lat_formatter=formatter, lng_formatter=formatter).add_to(m)
for x in range(len(result)):
    if x == 0:
        folium.Marker([result[x][0], result[x][1]], popup='Startport : %s'%start_port).add_to(m)
    elif x == len(result) - 1:
        folium.Marker([result[x][0], result[x][1]], popup='Endport : %s'%end_port).add_to(m)
    else:
        folium.CircleMarker([result[x][0], result[x][1]], radius=3, color='red', fill_color='red').add_to(m)
m

POLYGON ((-8.665589565454809 27.65642588959236, -8.665124477564191 27.58947907155823, -8.684399786809053 27.39574412689601, -8.6872936670174 25.88105621998891, -11.96941891117116 25.93335276946827, -11.93722449385332 23.37459422453617, -12.87422156416958 23.28483226164518, -13.11875444177471 22.77122020109626, -12.92910193526353 21.32707062426756, -16.84519365077399 21.33332347257488, -17.06342322434257 20.99975210213083, -17.02042843267577 21.42231028898158, -17.00296179856109 21.42073415779658, -14.75095455571353 21.50060008390366, -14.63083268885107 21.8609398462749, -14.22116777185725 22.31016307218816, -13.89111039880905 23.6910090194593, -12.50096269372537 24.7701162785782, -12.03075883630163 26.03086619720307, -11.71821977380036 26.10409170176062, -11.39255489749701 26.88342397715439, -10.55126257978527 26.99080760345689, -10.18942420087758 26.86094472910741, -9.735343390328879 26.86094472910741, -9.41303748212448 27.08847606048857, -8.794883999049077 27.12069631602251, -8.81782